# Lab Exercise Week 08 , Naive Bayes Algorithm

In [1]:
import pandas as pd
import requests
import io

url="https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data"
s=requests.get(url).content
credit=pd.read_csv(io.StringIO(s.decode('utf-8')), header=None)

In [2]:
credit.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


In [3]:
credit.shape

(690, 16)

In [4]:
# data preprocessing

In [5]:
import numpy as np
credit=credit.replace('?', np.nan)
credit=credit.dropna()
print(credit.shape)
credit.head(5)

(653, 16)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


In [6]:
credit.dtypes

0      object
1      object
2     float64
3      object
4      object
5      object
6      object
7     float64
8      object
9      object
10      int64
11     object
12     object
13     object
14      int64
15     object
dtype: object

In [7]:
# some types are not proper. we do type casting

In [8]:
credit.iloc[:,1]= credit.iloc[:,1].astype('float64')
credit.iloc[:,2]= credit.iloc[:,2].astype('float64')
credit.iloc[:,7]= credit.iloc[:,7].astype('float64')
credit.iloc[:,10]= credit.iloc[:,10].astype('float64')
credit.iloc[:,13]= credit.iloc[:,13].astype('float64')
credit.iloc[:,14]= credit.iloc[:,14].astype('float64')

credit.dtypes

0      object
1     float64
2     float64
3      object
4      object
5      object
6      object
7     float64
8      object
9      object
10    float64
11     object
12     object
13    float64
14    float64
15     object
dtype: object

In [9]:
credit.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1.0,f,g,202.0,0.0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6.0,f,g,43.0,560.0,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0.0,f,g,280.0,824.0,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5.0,t,g,100.0,3.0,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0.0,f,s,120.0,0.0,+


##  Design NB algorithm to predict the class of following sample where the sample must be predicted as negative (-)

In [25]:
x= credit.iloc[99,:]
np.array(x)

a1 = np.array([[1,2],[3,4]])
a2 = np.array([[10,20],[30,40]])

np.c_[a1,a2]


array([[ 1,  2, 10, 20],
       [ 3,  4, 30, 40]])

## Apply Naive Bayes Algorithm

Probability of that x belongs to Class-i (either + or -)is compıuted as follows, Naive Bayes Classifier

$$ P(C=i| x) = \Pi_{j=1}^{d} p(x_{j}| C=i)$$

For categorical variable, probability of P(x-j == value | C=i) is simply computed as # x-j == value / sample-size of C-i subset. However, for numerical variable, we need to compute mean and the standard deviation of the variable-j first. Further we can compute how likely a given numeric value occurs in the subset distribution of variable-j (x-j) given C-i as follows.

$$ P(x_{j}= value| C_{i})= \frac{1}{\sqrt{2\pi \sigma_{ij}^2}}  \epsilon^{- \frac{ (value -\mu_{ij})^2}{2 \sigma_{ij}^{2} }}$$
   

In [58]:
def probC_cat(ds, idx1_col, idx1_val, idx2_col, idx2_val):
    ds_idx2 = ds[ds[idx2_col] == idx2_val]
    
    return len(ds_idx2[ds_idx2[idx1_col] == idx1_val]) / len(ds_idx2)
    

In [59]:
def probC_val(ds, idx1_col, idx1_val, idx2_col, idx2_val):
    ds_idx2 = ds[ds[idx2_col] == idx2_val]
    idx1_mean = np.mean(ds_idx2[idx1_col])
    idx1_std = np.std(ds_idx2[idx1_col])
    
    return (1/np.sqrt(2*np.pi*(idx1_std**2)))*np.exp(-((idx1_val - idx1_mean)**2)/(2*idx1_std**2))

In [60]:
def probc(dataset, col_x, val_x, col_y, val_y):
    if (dataset.dtypes[col_x] == 'object'):
        retval =  probC_cat(dataset, col_x, val_x, col_y, val_y)
    else:
        retval =  probC_val(dataset, col_x, val_x, col_y, val_y)
    return retval

In [63]:
def nb(dataset, x, col_class, val_class):
    prob = 1
    for i in range(len(x)):
        if (i != col_class):
            prob *= probc(dataset, i, x[i], col_class, val_class)
    
    return prob

In [61]:
probc(credit, 1, 27.83, 15, '+')

0.028134581673098836

In [65]:
nb(credit, x, 15, '-')

1.7951479505991175e-16

In [66]:
import numpy as np
import math

#probablity of val_x in col_x of the dataset
def prob(dataset, col_x,  val_x):
    return len(dataset[dataset[col_x] == val_x]) / len(dataset)

#joint probability of val_x in col_x and val_y in col_y
def probJ(dataset, col_x, val_x, col_y, val_y):
    q = dataset[(dataset[col_x] == val_x) & (dataset[col_y] == val_y)]
    return (len(q) / len(dataset))

#conditional probability of val_x in col_x given val_y in col_y
def probC(dataset, col_x, val_x, col_y, val_y):
    
    if (dataset.dtypes[col_x] == 'object'):
        retval =  probC_cat(dataset, col_x, val_x, col_y, val_y)
    else:
        retval =  probC_val(dataset, col_x, val_x, col_y, val_y)
    return retval





#conditional probability (categorical) of val_x in col_x given val_y in col_y
def probC_cat(dataset, col_x, val_x, col_y, val_y):
    return probJ(dataset, col_x, val_x, col_y, val_y) / prob(dataset, col_y, val_y)

# method 2 -- conditional probability (categorical) of val_x in col_x given val_y in col_y
def probC_cat2(dataset, col_x, val_x, col_y, val_y):
    q = dataset[dataset[col_y] == val_y]
    return (len(q[q[col_x] == val_x]) / len(q))

#conditional probability (numerical) of val_x in col_x given val_y in col_y
def probC_val(dataset, col_x, val_x, col_y, val_y):
    q = dataset[dataset[col_y] == val_y]
    mean_col_x = np.mean(q[col_x])
    std_col_x = np.std(q[col_x])
    
    return (1 / (math.sqrt(2*np.pi*std_col_x**2)))*math.exp(-1*((val_x - mean_col_x)**2)/(2*std_col_x**2))





#return P(C = val_class|x)
def naive_bayes_classifier(dataset, col_class, val_class, x):
    prob =  1
    for i in range(len(x)):
        if (i != col_class):
            prob = prob*(probC(dataset, i, x[i], col_class, val_class))       
    return prob
    

In [67]:
#compare naive_bayes_classifier for '+' and '-' class.
#choose the class that has the higher probability, in this case '-' has a higher value.

print('class - :', naive_bayes_classifier(credit, 15, '-', x))
print('class + :', naive_bayes_classifier(credit, 15, '+', x))

class - : 1.7951479505991197e-16
class + : 6.262042529768192e-17


In [52]:
#check all records
count_corr = 0
for i in range(len(credit)):
    x = credit.iloc[i,:]
    prob_neg = naive_bayes_classifier(credit, 15, '-', x)
    prob_pos = naive_bayes_classifier(credit, 15, '+', x)
    if ((prob_neg > prob_pos) and (x[15] == '-')):
        count_corr = count_corr + 1
    elif ((prob_neg < prob_pos) and (x[15] == '+')):
        count_corr = count_corr + 1

print(str(count_corr / len(credit)))

0.7856049004594181


In [ ]:
#experimenting

In [ ]:
probC_val(credit, 1, 33.83, 15, '-')

In [53]:
probC(credit, 0, 'a', 15, '+')*probC(credit, 3, 'u', 15, '+')

0.2699849342585829

In [ ]:
probC(credit, 0, x[0], 15, '-')*probC(credit, 3, x[3], 15, '-')*probC(credit, 4, x[4], 15, '-')

In [ ]:
print(len(credit[(credit[15] == '+') & (credit[0] == 'b')]) / len(credit))

In [ ]:
q = credit[credit[15] == '+']
p = q[q[0] == 'b']
print(len(q))
print(len(p))

len(p)/len(q)

In [ ]:
probC_cat2(credit, 0, 'b', 15, '+')

In [ ]:
np.pi


In [ ]:
pos = credit[(credit[15] == '+')]
len(pos[pos[0] == 'b']) / len(pos)


np.var(pos[1])


In [ ]:
type(123.123)

In [ ]:
type(credit[15][1])

In [ ]:
if (credit.dtypes[0] == 'object'): 
    print(True)